In [ ]:
import pandas as pd
import numpy as np
import pathlib 
import os
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm import tqdm
import warnings
import pandas as pd
import argparse

from dataset_paths import *

warnings.filterwarnings('ignore')

PROJECT = "combine_ctcandi_ichorcna"
thres_hypo = 0.3
thres_hyper = 0.6

output_version = "20241229";
outdir = "/media/hieunguyen/HNSD_mini/outdir";

for mode in ["all", "hypo", "hyper"]:
    if mode == "all":
        path_to_03_output = os.path.join(outdir, PROJECT, output_version, "03_output")
    elif mode == "hypo":
        path_to_03_output = os.path.join(outdir, PROJECT, output_version, "03_output_all_hypo")
    elif mode == "hyper":
        path_to_03_output = os.path.join(outdir, PROJECT, output_version, "03_output_all_hyper")
    else: 
        raise ValueError("mode must be either all, hypo or hyper")

    all_countDMP_results = [item for item in pathlib.Path(path_to_03_output).glob("*/countDMPs.xlsx")]
    # remove gastric results, not statistically significant
    all_countDMP_results = [item for item in all_countDMP_results if "Gastric" not in str(item)]

    testdf = pd.DataFrame()

    for item in all_countDMP_results:
        tmpdf = pd.read_excel(item)
        tmpdf["Label"] = str(item).split("/")[-2]
        testdf = pd.concat([testdf, tmpdf], axis = 0)
    print("Number of TMD450 regions that have been tested by TCGA data: {}".format(testdf.shape[0]))
    if "hyper" not in testdf.columns:
        testdf["hyper"] = 0
    if "hypo" not in testdf.columns:
        testdf["hypo"] = 0

    testdf["hypo_or_hyper"] = testdf[["hyper", "hypo"]].apply(lambda x: "hyper" if x[0] > x[1] else "hypo", axis = 1)
    testdf["region_combiname"] = testdf[["Var1", "hypo_or_hyper"]].apply(lambda x: f"{x[0]}_{x[1]}", axis = 1)
    os.system(f"mkdir -p {path_to_03_output}/pan_cancer")
    testdf.to_excel(os.path.join(path_to_03_output, "pan_cancer", "countDMPs.xlsx"), index = False)

In [2]:
import pandas as pd
df = pd.read_excel("/media/hieunguyen/HNSD01/src/combine_ctcandi_ichorcna/metadata/REPORT4.xlsx")
df.to_csv("/media/hieunguyen/HNSD01/src/combine_ctcandi_ichorcna/metadata/REPORT4.csv")